### Data Cleaning
#### The purpose of this notebook is to create cleaned .csv files to export for use in my data analyses

More information about this project is available in my github repo here: https://github.com/Noah-Baustin/sf_crime_data_analysis

In [1]:
#import modules
import pandas as pd

/Users/nbaustin/.pyenv/versions/3.8.5/envs/sf_crime_data_analysis-3.8.5/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
# import historical csv into a variable
historical_data = pd.read_csv('raw_data/SFPD_Incident_Reports_2003-May2018/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv', dtype=str)

# import newer csv into a variable
newer_data = pd.read_csv('raw_data/SFPD_Incident_Reports_2018-10.14.21/Police_Department_Incident_Reports__2018_to_Present(1).csv', dtype=str)

Trim the extra columns that we don't need from the historical data:

In [4]:
historical_data = historical_data[
    ['PdId', 'IncidntNum', 'Incident Code', 'Category', 'Descript',
       'DayOfWeek', 'Date', 'Time', 'PdDistrict', 'Resolution', 'X',
       'Y', 'location']
].reset_index(drop=True)

Change the column names in the historical data to match the API names in the newer data. The SFPD published a key that I used to translate the column names over, which can be found on pg two of this document: https://drive.google.com/file/d/13n7pncEOxFTWig9-sTKnB2sRiTB54Kb-/view?usp=sharing

In [5]:
historical_data.rename(columns={'PdId': 'row_id',
                                'IncidntNum': 'incident_number',
                                'Incident Code': 'incident_code',
                                'Category': 'incident_category',
                                'Descript': 'incident_description',
                                'DayOfWeek': 'day_of_week',
                                'Date': 'incident_date',
                                'Time': 'incident_time',
                                'PdDistrict': 'police_district',
                                'Resolution': 'resolution',
                                'X': 'longitude',
                                'Y': 'latitude',
                                'location': 'the_geom'
                               }, 
                       inplace=True)

In [6]:
historical_data

,row_id,incident_number,incident_code,incident_category,incident_description,day_of_week,incident_date,incident_time,police_district,resolution,longitude,latitude,the_geom
0,3114751606302,031147516,06302,LARCENY/THEFT,PETTY THEFT FROM A BUILDING,Sunday,09/28/2003,10:00,SOUTHERN,NONE,-120.5,90,POINT (-120.50000000000001 90)
1,5069701104134,050697011,04134,ASSAULT,BATTERY,Wednesday,06/22/2005,12:20,NORTHERN,NONE,-122.428223303176,37.7818959488603,POINT (-122.42822330317601 37.7818959488603)
2,6074729204104,060747292,04104,ASSAULT,ASSAULT,Saturday,07/15/2006,00:55,CENTRAL,NONE,-122.410672425337,37.799788690123,POINT (-122.41067242533701 37.799788690123)
3,7103536315201,071035363,15201,ASSAULT,STALKING,Tuesday,09/25/2007,00:01,TARAVAL,NONE,-122.458226300605,37.7413616001449,POINT (-122.458226300605 37.7413616001449)
4,11082415274000,110824152,74000,MISSING PERSON,MISSING ADULT,Saturday,09/24/2011,11:00,TARAVAL,LOCATED,-122.459172646607,37.7082001648459,POINT (-122.459172646607 37.7082001648459)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2129520,16099543005043,160995430,05043,BURGLARY,"BURGLARY OF RESIDENCE, UNLAWFUL ENTRY",Thursday,12/01/2016,17:30,NORTHERN,NONE,-122.42253647830613,37.7787958771466,POINT (-122.42253647830613 37.7787958771466)
2129521,16093783009031,160937830,09031,OTHER OFFENSES,"MONEY, PROPERTY OR LABOR, FRAUDULENTLY OBTAINING",Monday,11/14/2016,00:01,INGLESIDE,NONE,-122.44276988992716,37.71275870748528,POINT (-122.44276988992716 37.71275870748528)
2129522,17078486916110,170784869,16110,DRUG/NARCOTIC,POSSESSION OF HEROIN FOR SALES,Tuesday,09/26/2017,01:38,CENTRAL,"ARREST, BOOKED",-122.40586321263237,37.79802284521844,POINT (-122.40586321263237 37.79802284521844)
2129523,18014823304138,180148233,04138,ASSAULT,"BATTERY, FORMER SPOUSE OR DATING RELATIONSHIP",Saturday,02/24/2018,20:30,CENTRAL,"ARREST, BOOKED",-122.40330497576683,37.792727599465884,POINT (-122.40330497576683 37.792727599465884)


Now let's trim down the columns from the newer dataset so that we're only working with columns that match up to the old data. 

Note: there's no 'the geom' column, but the column 'point' seems to be equivelant. 

In [7]:
newer_data = newer_data[
    ['Row ID', 'Incident Number', 'Incident Code', 'Incident Category', 
     'Incident Description', 'Incident Day of Week', 'Incident Date', 'Incident Time', 
     'Police District', 'Resolution', 'Longitude', 'Latitude', 'Point']
].copy()

Change the column names in the newer dataset to match the API names of the columns. Doing this because the original column names have spaces, which could cause issues down the road.

In [8]:
newer_data.rename(columns={'Row ID': 'row_id',
                           'Incident Number': 'incident_number',
                           'Incident Code': 'incident_code',
                           'Incident Category': 'incident_category',
                           'Incident Description': 'incident_description',
                           'Incident Day of Week': 'day_of_week', 
                           'Incident Date': 'incident_date',
                           'Incident Time': 'incident_time',
                           'Police District': 'police_district',
                           'Resolution': 'resolution',
                           'Longitude': 'longitude', 
                           'Latitude': 'latitude',
                           'Point': 'the_geom' 
                               }, 
                       inplace=True)

In [9]:
newer_data

,row_id,incident_number,incident_code,incident_category,incident_description,day_of_week,incident_date,incident_time,police_district,resolution,longitude,latitude,the_geom
0,107597928150,216138427,28150,Malicious Mischief,"Malicious Mischief, Vandalism to Property",Wednesday,2021/09/29,12:59,Ingleside,Open or Active,NaN,NaN,NaN
1,103010326030,210295348,26030,Arson,Arson,Friday,2021/05/14,01:51,Bayview,Open or Active,-122.38737260846696,37.74425940578451,POINT (-122.38737260846696 37.74425940578451)
2,107606971000,216138659,71000,Lost Property,Lost Property,Monday,2021/03/01,08:00,Southern,Open or Active,NaN,NaN,NaN
3,102792271000,216049830,71000,Lost Property,Lost Property,Thursday,2021/04/01,12:00,Park,Open or Active,NaN,NaN,NaN
4,107592206244,216138552,06244,Larceny Theft,"Theft, From Locked Vehicle, >$950",Tuesday,2021/06/01,07:01,Southern,Open or Active,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
513212,99729103074,210041000,03074,Robbery,"Robbery, W/ Force",Friday,2021/01/15,20:00,Tenderloin,Cite or Arrest Adult,-122.41214750780264,37.78589292711297,POINT (-122.41214750780264 37.78589292711297)
513213,94911806361,200461010,06361,Larceny Theft,"Theft, Shoplifting, <$50",Saturday,2020/08/01,17:34,Richmond,Open or Active,-122.47616919036905,37.780475849410344,POINT (-122.47616919036905 37.780475849410344)
513214,93192005061,200330863,05061,Burglary,"Burglary, Warehouse, Forcible Entry",Monday,2020/06/01,01:55,Bayview,Open or Active,-122.38985954971992,37.75399971361786,POINT (-122.38985954971992 37.75399971361786)
513215,97231606244,200642975,06244,Larceny Theft,"Theft, From Locked Vehicle, >$950",Saturday,2020/10/24,17:00,Northern,Open or Active,-122.43321873935555,37.777490366353746,POINT (-122.43321873935555 37.777490366353746)


In [10]:
historical_data.columns

Index(['row_id', 'incident_number', 'incident_code', 'incident_category',
       'incident_description', 'day_of_week', 'incident_date', 'incident_time',
       'police_district', 'resolution', 'longitude', 'latitude', 'the_geom'],
      dtype='object')

In [11]:
newer_data.columns

Index(['row_id', 'incident_number', 'incident_code', 'incident_category',
       'incident_description', 'day_of_week', 'incident_date', 'incident_time',
       'police_district', 'resolution', 'longitude', 'latitude', 'the_geom'],
      dtype='object')

Now that our datasets have matching columns, let's merge them together. 

In [12]:
frames = [historical_data, newer_data]
all_data = pd.concat(frames)

The dataframe all_data now contains our combined dataset!

In [13]:
all_data.info()
all_data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2642742 entries, 0 to 513216
Data columns (total 13 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   row_id                object
 1   incident_number       object
 2   incident_code         object
 3   incident_category     object
 4   incident_description  object
 5   day_of_week           object
 6   incident_date         object
 7   incident_time         object
 8   police_district       object
 9   resolution            object
 10  longitude             object
 11  latitude              object
 12  the_geom              object
dtypes: object(13)
memory usage: 282.3+ MB


,row_id,incident_number,incident_code,incident_category,incident_description,day_of_week,incident_date,incident_time,police_district,resolution,longitude,latitude,the_geom
0,3114751606302,031147516,06302,LARCENY/THEFT,PETTY THEFT FROM A BUILDING,Sunday,09/28/2003,10:00,SOUTHERN,NONE,-120.5,90,POINT (-120.50000000000001 90)
1,5069701104134,050697011,04134,ASSAULT,BATTERY,Wednesday,06/22/2005,12:20,NORTHERN,NONE,-122.428223303176,37.7818959488603,POINT (-122.42822330317601 37.7818959488603)
2,6074729204104,060747292,04104,ASSAULT,ASSAULT,Saturday,07/15/2006,00:55,CENTRAL,NONE,-122.410672425337,37.799788690123,POINT (-122.41067242533701 37.799788690123)
3,7103536315201,071035363,15201,ASSAULT,STALKING,Tuesday,09/25/2007,00:01,TARAVAL,NONE,-122.458226300605,37.7413616001449,POINT (-122.458226300605 37.7413616001449)
4,11082415274000,110824152,74000,MISSING PERSON,MISSING ADULT,Saturday,09/24/2011,11:00,TARAVAL,LOCATED,-122.459172646607,37.7082001648459,POINT (-122.459172646607 37.7082001648459)


We need to convert our incident_date column into a datetime format

In [14]:
all_data['incident_date'] = pd.to_datetime(all_data['incident_date'])

In [15]:
all_data['incident_date'].min()

Timestamp('2003-01-01 00:00:00')

In [16]:
all_data['incident_date'].max()

Timestamp('2021-10-13 00:00:00')

We can see from the date max and min that we've got our full set of date ranges from 2003 to 2021 in this new combined dataframe.

Since our string search we'll need to pull out our marijuana cases is cap sensitive, let's put all our values in the incident_description in lowercase:

In [19]:
all_data['incident_description'] = all_data['incident_description'].str.lower()

Now let's create a dataframe with all our marijuana data:

In [20]:
all_data_marijuana = all_data[
    all_data['incident_description'].str.contains('marijuana')
].reset_index(drop=True)

In [21]:
all_data_marijuana

,row_id,incident_number,incident_code,incident_category,incident_description,day_of_week,incident_date,incident_time,police_district,resolution,longitude,latitude,the_geom
0,16055139916010,160551399,16010,DRUG/NARCOTIC,possession of marijuana,Friday,2016-07-08,08:00,MISSION,"ARREST, BOOKED",-122.42326589360349,37.765649515945,POINT (-122.42326589360349 37.765649515945)
1,17102985016010,171029850,16010,DRUG/NARCOTIC,possession of marijuana,Thursday,2017-12-21,10:40,TARAVAL,"ARREST, BOOKED",-122.45364594949392,37.72327255110331,POINT (-122.45364594949392 37.72327255110331)
2,17026584716010,170265847,16010,DRUG/NARCOTIC,possession of marijuana,Saturday,2017-04-01,02:10,NORTHERN,"ARREST, BOOKED",-122.43959183986001,37.783850873845424,POINT (-122.43959183986001 37.783850873845424)
3,16071288616010,160712886,16010,DRUG/NARCOTIC,possession of marijuana,Friday,2016-09-02,17:30,PARK,"ARREST, BOOKED",-122.45351291112611,37.76869697865512,POINT (-122.45351291112611 37.76869697865512)
4,16054757016030,160547570,16030,DRUG/NARCOTIC,possession of marijuana for sales,Wednesday,2016-07-06,18:32,RICHMOND,NONE,-122.46620466789287,37.772540539159316,POINT (-122.46620466789287 37.772540539159316)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22072,95820816010,200528905,16010,Drug Offense,marijuana offense,Wednesday,2020-09-02,16:28,Ingleside,Cite or Arrest Adult,-122.43129384491388,37.72872954740928,POINT (-122.43129384491388 37.72872954740928)
22073,97176416030,200638140,16030,Drug Offense,"marijuana, possession for sale",Thursday,2020-10-22,08:47,Taraval,Cite or Arrest Adult,-122.5040487777567,37.76426117163131,POINT (-122.5040487777567 37.76426117163131)
22074,99731116030,210041141,16030,Drug Offense,"marijuana, possession for sale",Tuesday,2021-01-19,03:13,Mission,Open or Active,-122.41811812009416,37.7603010605011,POINT (-122.41811812009416 37.7603010605011)
22075,99688016030,210037532,16030,Drug Offense,"marijuana, possession for sale",Sunday,2021-01-17,10:05,Park,Cite or Arrest Adult,-122.42746205880601,37.76877049785351,POINT (-122.42746205880601 37.76877049785351)


In [22]:
all_data_marijuana['incident_date'].min()

Timestamp('2003-01-01 00:00:00')

In [23]:
all_data_marijuana['incident_date'].max()

Timestamp('2021-10-09 00:00:00')

The incident dates show that we're getting marijuana incidents from 2003 all the way up to 2021. Great!

Let's export our two dataframes to .csv's that we can now use in other data analysis!

In [24]:
all_data.to_csv("all_data.csv", index=False)
all_data_marijuana.to_csv("all_data_marijuana.csv", index=False)